# Lotus Chorus Workshop 

>*a* γυμνάσιον

>*for Helen Zell*

To run this notebook, select from above ***Runtime>Run all***. The interactive dialogue will appear at the bottom of the notebook.

In [1]:
%%capture
pip install nltk==3.6.7

In [2]:
%%capture
import nltk
nltk.download('book')
nltk.download('omw-1.4')
from nltk import tokenize,pos_tag

In [3]:
%%capture
pip install spacy==3.2

In [4]:
%%capture
!python -m spacy download en_core_web_md

In [5]:
import spacy
nlp = spacy.load("en_core_web_md")

In [6]:
def toktag(text):
  """
  utility function to tokenize and pos_tag
  using spacy but in nltk tuple style
  """
  return [(i.text,i.tag_) for i in nlp(text)]

In [7]:
import random
import re
import itertools

In [8]:
def randrand(alist,min=1):
  """
  utility function to randomly sample
  a random number of elements from a list
  """
  n = random.randint(min,len(alist))
  return random.sample(alist,n)


randrand([1,2,3,4,5],min=3)

[5, 1, 4, 2, 3]

**Comment on noun:** template-based approach to commenting on a noun

Template includes words from WordNet.

In [9]:
simple_noun_recommendations = [#what if the <targetNN> were more <JJ>?",
                               'What about "<JJ> <targetNN>"? Or "<JJ> <targetNN>"?',
                               'In my opinion, the word "<targetNN>" is a bit too <JJneg>.',
                               'I feel like you could replace "<targetNN>" with something a bit more <JJpos>?',
                               #"what if the <targetNN> were <JJ>, maybe even like a <NN>?",
                               #Boring. What if you replaced "<targetNN>" with <NN>?'
                               #"'<targetNN>'?  What if this were the opposite?"
                               ]

In [10]:
from nltk.corpus import wordnet as wn

In [11]:
nouns = [i.lemma_names()[0].replace("_"," ") for i in list(wn.all_synsets('n'))]
len(nouns)

82115

In [12]:
adjectives = [i.lemma_names()[0].replace("_"," ") for i in list(wn.all_synsets("a"))]
len(adjectives)

18156

In [13]:
# myNN = ["wolf","person","machine","orbit","fancy","past","future","clothing","toy",
#         "idea","interiority","dream","path","virtue","sin","polital perspective","language","superego","ego","id","shampoo","education"]
# my

myNN = nouns#_nn_clean
myJJ = adjectives#_jj_clean
JJneg = ["florid","petite","grim","long",
           "trite","honeyed","brittle",
           "happy","postmodern","mainstream",
           "expected","quirky","soft","salty","anthropomorphic","academic",
           "North American","old","phallic","representational","abstract","polysemic","simple"]

JJpos = ["erudite","crisp","evocative","neutral","global","critical",
         "self-aware","ironic","careful","polysemous","deep","emotionally resonant","politically important","deserved","ambivalent",
         "contemporary","specific","true to your life","holy","holy","holy","reverent"
         ]

tag2options = {"NN":myNN,"JJ":myJJ,"JJneg":JJneg,"JJpos":JJpos}

In [14]:
def generate_simple_noun_rec():
  template = random.choice(simple_noun_recommendations)
  to_replace = [t for t in re.findall(r'(?:<)([A-Za-z]{1,})(?:>)',template) if t.startswith('target')==False]
  for tag in to_replace:
    template = template.replace("<"+tag+">",random.choice(tag2options[tag]),1)
  return template


generate_simple_noun_rec()

'What about "sexless <targetNN>"? Or "sad <targetNN>"?'

In [15]:
def COMMENT_NN_recommendation(line):
  pos_tagged = toktag(line)
  nouns = [token for token,tag in pos_tagged if tag=="NN"]
  if len(nouns)<1:
    return
  noun = random.choice(nouns)
  template = generate_simple_noun_rec()
  template = template.replace("<targetNN>",noun.lower())
  return template
  
COMMENT_NN_recommendation("The dog is in the cave.")

'I feel like you could replace "dog" with something a bit more polysemous?'

**Comment on noun via Wikipedia:** suggests a related rare/specific word from the key term's Wikipedia page.

In [16]:
from bs4 import BeautifulSoup
import requests

In [17]:
page = requests.get("https://en.wikipedia.org/wiki/Skin")
soup = BeautifulSoup(page.text, 'html.parser')


In [18]:
wikitext = " ".join([s.getText() for s in soup(["p"])])

In [19]:
wikitext = wikitext.replace("\n"," ")
wikitext

'Skin is the layer of usually soft, flexible outer tissue covering the body of a vertebrate animal, with three main functions: protection, regulation, and sensation.[1]  Other animal coverings, such as the arthropod exoskeleton, have different developmental origin, structure and chemical composition. The adjective cutaneous means "of the skin" (from Latin cutis \'skin\'). In mammals, the skin is an organ of the integumentary system made up of multiple layers of ectodermal tissue and guards the underlying muscles, bones, ligaments, and internal organs. Skin of a different nature exists in amphibians, reptiles, and birds.[2] Skin (including cutaneous and subcutaneous tissues) plays crucial roles in formation, structure, and function of extraskeletal apparatus such as horns of bovids (e.g., cattle) and rhinos, cervids\' antlers, giraffids\' ossicones, armadillos\' osteoderm, and os penis/os clitoris.[3]  All mammals have some hair on their skin, even marine mammals like whales, dolphins, 

In [20]:
def get_wiki_text(word):
  page = requests.get("https://en.wikipedia.org/wiki/%s" % word)
  soup = BeautifulSoup(page.text, 'html.parser')
  wikitext = " ".join([s.getText() for s in soup(["p"])])
  return wikitext

wiki_text = get_wiki_text("skin")
wiki_text[:10]

'Skin is th'

In [21]:
wiki_text[-1000:]

" is the J-curve stress strain response, in which a region of large strain and minimal stress exists and corresponds to the microstructural straightening and reorientation of collagen fibrils.[33] In some cases the intact skin is prestreched, like wetsuits around the diver's body, and in other cases the intact skin is under compression. Small circular holes punched on the skin may widen or close into ellipses, or shrink and remain circular, depending on preexisting stresses.[34]\n Tissue homeostasis generally declines with age, in part because stem/progenitor cells fail to self-renew or differentiate. Skin aging is caused in part by TGF-β by blocking the conversion of dermal fibroblasts into fat cells which provide support. Common changes in the skin as a result of aging range from wrinkles, discoloration, and skin laxity, but can manifest in more severe forms such as skin malignancies.[35][36] Moreover, these factors may be worsened by sun exposure in a process known as photoaging.[36

In [22]:
from nltk.corpus import gutenberg,brown
gutenwords = list(set([w.lower() for w in gutenberg.words()]))
brownwords = list(set([w.lower() for w in brown.words()]))
typical_words = gutenwords+brownwords

In [23]:
def get_words_from_wiki_text(wikitext,min_n=3):
  words = toktag(wikitext)
  possible_words = [token.lower() for token,tag in words if tag in ["NN","NNS","JJ"] and token.lower() not in typical_words]
  possible_words = [w for w in possible_words if w.isalpha() and len(w)>=5]
  possible_words_at_least_n = [w for w in possible_words if possible_words.count(w)>=min_n] ## at least
  return possible_words_at_least_n

get_words_from_wiki_text(wiki_text)[:20]

['cutaneous',
 'amphibians',
 'cutaneous',
 'subcutaneous',
 'amphibian',
 'amphibian',
 'basal',
 'keratinocytes',
 'keratinocytes',
 'keratinocytes',
 'keratin',
 'extracellular',
 'keratinocytes',
 'dermis',
 'dermis',
 'dermis',
 'dermis',
 'dermis',
 'extracellular',
 'dermis']

In [24]:
def get_related_wikipedia_words(text):
  words = toktag(text)
  possible_words = list(set([token.lower() for token,tag in words if tag in ["NN","NNP"] and token.isalpha()]))
  random.shuffle(possible_words)
  for pw in possible_words:
    try:
      relateds = get_words_from_wiki_text(get_wiki_text(pw))
      if relateds!=[]:
        return (pw,relateds)
    except:
      pass

#get_related_wikipedia_words("The skin is king")

In [25]:
def COMMENT_with_wikipedia(text):
  word_and_relateds = get_related_wikipedia_words(text)
  if word_and_relateds==None:
    return
  word,related_words = word_and_relateds
  rw = random.choice(related_words)
  pizzazz = ['pizzazz','oomph','cleverness','more interesting language','specificity']
  return 'This needs some %s...when I see the word "%s" I think of "%s."' % (random.choice(pizzazz),word,rw)


COMMENT_with_wikipedia("the skin is a dog.")

'This needs some specificity...when I see the word "skin" I think of "cutaneous."'

**Comment on chunk:** Extract a syntactic chunk (phrase, clause, etc.) from input text and comment on it.

In [26]:
%%capture
pip install benepar

In [27]:
import benepar
benepar.download('benepar_en3')

True

In [28]:
nlp.add_pipe("benepar", config={"model": "benepar_en3"})

In [29]:
def get_chunks_from_a_string(inputstring):
  all_chunks = []
  top_chunks = list(list(nlp(inputstring).sents)[0]._.children)
  #all_chunks += top_chunks
  stack = top_chunks
  while True:
    try:
      current_string = stack.pop(0)
      all_chunks.append(current_string)
      chunks = list(current_string._.children)
      if len(list(chunks))!=0:
        all_chunks += chunks
        stack += chunks
    except:
      return all_chunks


list(get_chunks_from_a_string("That old wallet picture is not in the really old house where I sleep."))

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/distributions/distribution.py:51: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


[That old wallet picture,
 That,
 old,
 wallet,
 picture,
 is not in the really old house where I sleep,
 is,
 not,
 in the really old house where I sleep,
 .,
 That,
 old,
 wallet,
 picture,
 is,
 not,
 in the really old house where I sleep,
 in,
 the really old house where I sleep,
 in,
 the really old house where I sleep,
 the really old house,
 where I sleep,
 the really old house,
 the,
 really old,
 house,
 where I sleep,
 where,
 I sleep,
 the,
 really old,
 really,
 old,
 house,
 where,
 I sleep,
 I,
 sleep,
 really,
 old,
 I,
 sleep]

In [30]:
import warnings
warnings.filterwarnings("ignore")

In [31]:
positives = ["specific","erudite","clear","crisp",\
             "noble","modern","colorful","humane",\
             "connected","tender"]

negatives = ["over the top","verbose","'poetic'","expected","macho","effete"]

topics = ['flowers','dirt',"moisture","kind of classical allusion","toxic energy","alliteration","rhythm"]
topics_JJ = ["reflective of "+i for i in ["our political situation","the climate catastrophe","globalization","our digitalial millieu"]]
concepts = ['truth','religion','gender','economics','communism','markets','futurity','evil','organization','chaos','technology','consciousness','embodiment','salvation','prayer']
topics_JJ += ["reflective of the %s between %s and %s" % (random.choice(['link','tension']),a,b) for a,b in list(itertools.permutations(concepts,2))]

simple_chunk_recommendations = [
                                "'<targetChunk>'...don't you think this is a little too <negative>?",
                                "'<targetChunk>'...I just wish this could be a little more <topicJJ>",
                                "'<targetChunk>'...I just feel like this needs some <topic>?",
                                "'<targetChunk>'---this is the soul of the line.  (Everything else...I don't love?)",
                                "'<targetChunk>'---I'd try something else here.",
                                "'<targetChunk>'---yes, I want this but more so---dial it up!"
                                ]


In [32]:
def generate_simple_chunk_rec():
  tag2options = {"topic":topics,"negative":negatives,"topicJJ":topics_JJ}
  template = random.choice(simple_chunk_recommendations)
  to_replace = [t for t in re.findall(r'(?:<)([A-Za-z]{1,})(?:>)',template) if t.startswith('target')==False]
  for tag in to_replace:
    template = template.replace("<"+tag+">",random.choice(tag2options[tag]),1)
  return template

def COMMENT_chunk_recommendation(text):
  all_chunks = get_chunks_from_a_string(text)
  ok_chunks = [c for c in all_chunks if len(c)>2 and len(c)<9]
  if len(ok_chunks)<1:
    return
  chunk = random.choice(ok_chunks).text
  template = generate_simple_chunk_rec()
  template = template.replace("<targetChunk>",chunk)
  return template
  
COMMENT_chunk_recommendation("I like you very much, but under the swollen forest there is no hope")

"'is no hope'...don't you think this is a little too expected?"

**Ban Topics**: Forbid topics.

In [33]:
import gensim.downloader as api

In [34]:
vector_model = api.load("glove-wiki-gigaword-50")

[=================================================-] 99.0% 65.3/66.0MB downloaded

In [35]:
topic_words = {
    "nature":["rock","tree","flower","dirt","sun","sky","stone","soil","grass","moss"],
    "animal":["wolf","spider","hamster","cat","dog","hyena","rat","lion","bear","koala","snake","bug","deer"],
    "bird":["bird","penguin","owl","nest","egg","beak","feather","raven","flight","roost"],
    "celestial":["moon","sky","sun","planet","constellation","star","pisces","capricorn","zodiac"],
    "digital":["computer","twitter","facebook","phone","iphone","app","hacker","algorithm","code"],
    "religion":["angel","god","pray","prayer","salvation","sin","saint","forgiveness","fasting","resurrection","liturgy","grace"],
    "school":["university","college","exam","quiz","student","study","degree"],
    "violent":["sword","gun","missile","knife","kick","punch","blood","break","cut","die","laceration","injury"],
    "kinfolk":["father","brother","mother","cousin","family"],
    "political":["communism","democracy","government","vote","election","coup","anarchy","fascism","leader","president","senator"],
    "body":["spit","shit","fart","vomit","bone","heart","lung","tongue","lymph","blood","joint"],
    "war":["bombing","invastion","incursion","army","soldier","casualty","tank","artillery"],
    "ocean":["seaweed","whale","ocean","brine","deepwater","atlantic","pacific","sea","turtle","wave","tide"],
    "industrial":["factory","machine","forge","electricity","gear","dynamo","steel","coal","mill","product","industrialization","sweatshop"],
    "finance":["trade","market","dollar","bank","investment","finance","capitalism","cash","gold","silver"],
    "science":["experiment","element","chemical","microscope","scientist","observatory","research","physics","oxygen","helium","hydrogen","isotope","atomic",],
    "writing":["pen","pencil","paper","book","poem","essay","literature"],
    "darkness":["dark","night","dusk","midnight","moon","gloaming"],
    "lustrous":["light","shine","brightness","beam","sun","dawn"],
    "sport":["sport","racquet","ball","baseball","football","tennis","basketball","throw","catch"],
    "romance":["romance","beloved","adore","love","passion","sex","kiss","yearning"],
    "death":["dead","death","buried","funeral","demise","forgotten","grave","dying"],
    "face":["face","eye","mouth","expression","nose","smle","lips","forehead","frown"],
    "shopping":["shopping","purchase","deal","catalogue","sale","dollar","shop","mall"],
    "season":["season","winter","summer","spring","autumn"],
    "music":["music","song","orchestra","singer","melody","tune","ballad","concerto","violin","guitar","piano"],
    "law":["law","criminal","conviction","court","judge","tribunal","tort","constitution","prisoner"],
    "urban":["skyscraper","city","urban","subway","taxi","crowd","café","flaneur"],
    "food":["food","eat","taste","snack","lunch","dinner","delicious","meal","meat","sushi","pasta","bread"],
    "alcohol":["wine","beer","alcohol","drunk","cocktail","bourbon","vodka"],
    "furniture":["chair","table","sofa","clock","drawer"],
    "architectural":["architecture","roof","blueprint","building","skyscraper","door","window","mansion","brownstone"],
    "negative":["sadness","envy","scream","hate","insomnia","loss","bereft","emptiness","frustration","weep"],
}

all_topics = topic_words.keys()

In [36]:
all_topics = randrand(all_topics,min=round(len(all_topics)*.5)) ## sample
len(all_topics)

28

In [37]:
def sim(w1,w2,model=vector_model):
  """
  error handle list comprehension
  """
  try:
    return model.similarity(w1,w2)
  except:
    pass


def test_word_against_topic_sets(word,topics=all_topics,thresh=0.67,top_n=4):
  related_topic = None
  related_score = -100
  for topic in topics:
    related_words = topic_words[topic]
    scored = [s for s in [sim(word,rw) for rw in related_words] if s!=None] ## filter out nones from sim()
    scored.sort(reverse=True)
    if len(scored)>0: ## stop zero division error
      top_avg = (sum(scored[:top_n])/len(scored[:top_n]))
      if (top_avg>related_score) and (top_avg>thresh):
        related_score = top_avg
        related_topic = topic
  return related_topic
    
test_word_against_topic_sets('house',topic_words)

'architectural'

In [38]:
import random
n_topics_to_ban = random.randint(1,len(topic_words))

In [39]:
banned = random.sample(topic_words.keys(),n_topics_to_ban)
banned

['industrial',
 'body',
 'face',
 'religion',
 'writing',
 'finance',
 'political',
 'shopping',
 'lustrous',
 'darkness',
 'music',
 'romance',
 'death',
 'sport',
 'school',
 'furniture',
 'kinfolk',
 'nature',
 'food',
 'digital',
 'negative',
 'urban',
 'violent',
 'ocean',
 'bird',
 'celestial',
 'architectural',
 'war',
 'science']

In [40]:
def COMMENT_test_against_banned_topics(text,banned_topics=banned):
  pos_tagged = toktag(text)
  nouns = [token.lower() for token,tag in pos_tagged if tag in ["NN","NNS"]]
  random.shuffle(nouns)
  for n in nouns:
    banned = test_word_against_topic_sets(n,topics=banned_topics)
    if banned!=None:
      hmm = random.choice(['hmm','ugh','really?'])
      que = random.choice([".",'?',", maybe?",", no?","!"])
      dam = random.choice(["damn ","tedious ","","","",""])
      return '"%s"?...%s...enough of this %s%s stuff%s' % (n.capitalize(),hmm,dam,banned,que)
      break

COMMENT_test_against_banned_topics("I believe in the silver.")

**Ban Letter**: Forbid the use of a letter that occurs in the input string

In [41]:
def COMMENT_remove_letter(text):
  try:
    letter = random.choice([l for l in list(set([l for l in text if l.isalpha()])) if text.count(l)>3])
    return 'Try getting rid of all the "%s"\'s.%s' % (letter,random.choice([" There are entirely too many.",""]))
  except IndexError:
    pass

COMMENT_remove_letter("This is so deliciously small.")

'Try getting rid of all the "i"\'s. There are entirely too many.'

**Begins With:** Tells writer to replace a word with another word beginning with a specific letter with a specific number of syllables and possibly a source (e.g. Milton or Dickinson)

In [42]:
from nltk.corpus import stopwords
stops = stopwords.words('english')

In [43]:
sources = ["Milton","Emily Dickinson","an LL Bean catalogue","a set of instructions","an old architectural magazine from Japan",
             "George Herbert","a conversation with a dear one","your private language","how your friends used to talk",
             "a family record", "a legal matter","a secret internet","the Bible","the Apophthegmata","the Baltimore Catechism",
             "Speech and Language Processing 2nd Edition (Jurafsky & Martin)","a witty conversation","a psychoanalytic seminar","an argument"]
sources+=random.sample(["an old book about how a %s works" % w for w in ["company","market","body","family","capitalist economy",
                                                                           "friendship","thought","mechanical watch","confession",
                                                                          "miracle","city","language","computer","syndicalist economy",
                                                                           "prison","superego","mezzotint","monastery"]],3)
  

sources = randrand(sources,5)

In [44]:
def letter_comment(word):
  alphabet = 'abcdefghijklmnopqrstuv'
  letter = random.choice(alphabet.replace(word[0].lower(),"")) ## not already first letter
  return 'with a word that begins with "%s"' % letter

def syllable_comment(comment):
  n = random.randint(1,6)
  comment = comment.replace("with a word","with a %d syllable word" % n)
  return comment

def source_comment():
  source = random.choice(sources)
  leadup = random.choice(['sounds like it could be from','puts one in mind of','has the flavor of'])
  return "and that %s %s" % (leadup,source)

def comment_on_word(word):
  comment = 'What if you swapped "%s"' % word
  #if random.choice([True,True,True,True,False]):
  comment+=" "+letter_comment(word)
  if random.choice([True,False]):
    comment = syllable_comment(comment)
    if random.choice([True,False]):
      comment+=" "+source_comment()
  # else:
  #   comment+=" with a %s word" % random.choice(['better','sweeter','greener','redder','bluer','indestructable','truer','more costly','holier'])
  comment = comment+'?' ## add period
  return comment

def COMMENT_replace_word(text):
  pos_tagged = toktag(text)
  possibilities = [token for token,tag in pos_tagged if (tag[0] in "NJV" and token.lower() not in stops)]
  if len(possibilities)<1:
    return
  to_comment_on = random.choice(possibilities)
  return comment_on_word(to_comment_on)

COMMENT_replace_word("so this is my name")

'What if you swapped "name" with a 4 syllable word that begins with "h" and that sounds like it could be from a witty conversation?'

**Flip:** Suggest alternate starting word from other words in sentence.

In [45]:
def COMMENT_flip(text):
  tokens = [w.lower() for w in tokenize.word_tokenize(text)]
  possible_first_words = [w for w in tokens if w!=tokens[0] and w.isalpha() and w not in stops]
  if len(possible_first_words)<1:
    return
  new_first_word = random.choice(possible_first_words)
  return 'I\'d move "%s" to the beginning of the sentence.' % new_first_word

COMMENT_flip("The hill is not in the world.")

'I\'d move "world" to the beginning of the sentence.'

**Forbid Part of Speech**: Ban a part of speech that the input text contains.

In [46]:
def COMMENT_ban_pos(text):
  tag2pos = {"JJ":"adjectives","IN":"prepositions","DT":"determiners (or 'articles') like 'an' or 'the' etc.",
             "NN":"nouns","NNS":"nouns","RB":"adverbs","NNP":"proper nouns","NNPS":"proper nouns",
             "VB":"verbs","VBG":"gerunds or present participles",
           "SYM":'symbols (e.g. "$")',
           "VBN":"past participles","VBZ":"present tense verbs","VBD":"past tense verbs","PRP":"pronouns"}
  ### only rarely forbid certain ones
  if random.choice([True,True,True,True,False]):
    del tag2pos["DT"]
  if random.choice([True,True,True,True,False]):
    del tag2pos["NN"]
    del tag2pos["NNS"]
  if random.choice([True,True,False]):
    del tag2pos["PRP"]
  tags = [tag for token,tag in toktag(text)]
  tags_possible_to_ban = [t for t in tags if t in tag2pos]
  if tags_possible_to_ban == []:
    return ## none
  tag_to_ban = random.choice(tags_possible_to_ban)
  return "Try this again but without any %s." % tag2pos[tag_to_ban]

COMMENT_ban_pos("Running won't save water from falling.")

'Try this again but without any verbs.'

**Proper Noun/Location:** Requests that a noun be changed to a proper noun with reference to a specific country, city, etc.. 

In [47]:
places = ["Beijing","Shenzen","Brussels","Brentwood","Oakland",
          "Carpathian Ruthenia","upstate","London","Japan","Mexico", "the Great Steppe",
          "Argentina","Uruguay","New Jersey","Northern California",
          "Nashville","Ancient Greece","Ancient Rome","the USSR",
          "Nigeria","Iraq","Nepal","France","French Canada","Finland",
          "Russia","Oslo","middle America","Byzantium","Byzantium","Illyria","Christendom"]

places = randrand(places,min=5) ## sample

def noun_to_proper_noun_request(noun):
  place = random.choice(places)
  return_string = "Too vague. Make this a specific %s from %s." % (noun,place)
  return return_string

def COMMENT_solicit_proper_noun(text):
  pos_tagged = toktag(text)
  nn_s = [token.lower() for token,tag in pos_tagged if tag=="NN"]
  #nns_s = [token for token,tag in pos_tagged if tag=="NNS"]
  propers = [token for token,tag in pos_tagged if tag.startswith("NP")]
  if propers==[] and nn_s!=[]:
    noun = random.choice(nn_s)
    return noun_to_proper_noun_request(noun)

COMMENT_solicit_proper_noun("You are my friend and my dog.")

'Too vague. Make this a specific dog from French Canada.'

**Rhyme Suggestion:** Suggests that a word from input text rhyme or alliterate with another word from input text

In [48]:
def COMMENT_suggest_rhyme(text):
  pos_tagged = toktag(text)
  possible_words = [token for token,tag in pos_tagged if tag in ["NN","NNS","NNP","NNPS","JJ"]]
  if len(possible_words)<2:
    return
  w1,w2 = random.sample(possible_words,2)
  if random.choice([True,True,False]):
    if w1[-2:]!=w2[-2:]:  ## not a great way of detecting preexisting rhyme
      return 'Replace "%s" with a similar word that rhymes with "%s."' % (w1,w2)
  else:
    if w1[0]!=w2[0]: ## not a perfect way of detecting preexisting alliteration
      return 'Replace "%s" with a similar word that is alliterative with "%s."' % (w1,w2)

COMMENT_suggest_rhyme("A star fell in my soup container")

'Replace "container" with a similar word that is alliterative with "soup."'

**Prefixification:** Recommends a new version of a word with a prefix and/or suffix.

In [49]:
prefixes = ["hyper-","hypo-","un-","extra-","eigen","iso","thermo","uni","arche-",
            "de-","anti-","pre-","mega","micro","bio","epi","leio-","geo","gymno-","gyro-",
            "tele-","e-","post","re","centi-","hecto-","auto-","dys","counter-",
            "ante","ab","bi","co-","exo-","endo-","ecto-","mal","multi-","mono","intra","inter","cyber-",
            "pre","i-","Tex-","Euro-","pseudo-","fore","sub","proto","hemi","x-","xeno-"
            "thermo-","eco","nimbo-","aqua-","cosmo-","psycho-","crypto-","equi-","ultra","nocto-",
            "neo-","Russo-","Sino-","ur-","Mc","novo","malo","St.","Fitz","meso-","ova","zoa,"
            ]

prefixes = list(set(prefixes))

prefixes = randrand(prefixes,min=round(len(prefixes)*.8))

suffixes = ["ification","istan","burg","berg","ery","ship","osis","ology","ic","ic","ic","ogen","ivore",
            "hood","ism","ant","ful","ectomy"," de guerre","ite","licious","arium","itis","opathy","otrope",
            "ometry","ous","opoiesis","ospasm","oid","osphere","core","ette","-Con","-gate","fest",".com",".io",".ru","-cel"]

suffixes = list(set(suffixes))

suffixes = randrand(suffixes,min=round(len(suffixes)*.8))  

def COMMENT_fix_a_word(text):
  pos_tagged = toktag(text)
  poss_words = [(token.lower(),tag) for token,tag in pos_tagged if (tag=="NN")]#or tag.startswith("V") or tag.startswith("JJ"))]
  poss_words = [(token,tag) for token,tag in poss_words if token.lower() not in stops]
  if poss_words==[]:
    return ## no options
  word,tag = random.choice(poss_words)
  fixed = word
  if random.choice([True,True,False]):
    pre = random.choice(prefixes)
    if fixed.startswith(pre)==False:
      fixed = pre+fixed
    if (random.choice([True,False,False,False]) and tag=="NN"):
      suf = random.choice(suffixes)
      if fixed.endswith(suf)==False:
        fixed = fixed+suf
  else:
    if tag!="NN":
      return ## can't add suffix, return nothing
    suf=random.choice(suffixes)
    if fixed.endswith(suf)==False:
      fixed = fixed+suf
    if random.choice([True,False,False,False]):
      pre = random.choice(prefixes)
      if fixed.startswith(pre)==False:
        fixed = pre+fixed
  return 'Instead of "%s," what about "%s"?' % (word,fixed) 


COMMENT_fix_a_word("Crown in hand, my old hat, I ran through THE STORE")

'Instead of "hand," what about "bihand"?'

**Recommend Literary Device:** randomly suggest one or two literary devices, rhetorical figures, or other such things.

(Fallback function: does not take input.)

In [50]:
## base figs, from some "literary figures" website and my own reckoning or maybe "literary devices"

figures = ["aphaeresis","zeugma","chiasmus","foreign word","number","asyndeton","adynaton","anacoluthon",
                   "allusion","antimetabole","anadiplosis","anthimeria","catachresis","circumlocution","catalog",
                   "chiasmus","caesura","cliché","diacope","dysphemism","encomium","elision","ellipsis",
                   "enthymeme","enumeration","internal rhyme","kenning","malapropism","moral","neologism","oxymoron","onomatopoeia",
           "spoonersim","palindrome","pleonasm","pun","sibilance","solecism","swear word","brand name",
]

extra_figure_number = len(figures)//2

## from silva rhetoricae 
byu_figs = ["abating", "abbaser", "abecedarian", "abcisio", "ablatio", "abominatio", "abuse", "abusio", "abusion", "acoloutha", "accismus", "accumulatio", "accusatio adversa", "accusatio", "acervatio", "acrostic", "acyrologia", "acyron", "adage", "adagium", "addubitatio", "adhortatio", "adianoeta", "adjectio", "adjournment", "adjudicatio", "adjunct", "adjunctio", "admonitio", "adnexio", "adnominatio", "adynata", "adynaton", "aeschrologia", "aetiologia", "affirmatio", "affirmation", "aganactesis", "agnominatio", "agnomination", "aischrologia", "allegory", "alleotheta", "alliteration", "amar Airrisio", "ambiguitas", "ambiguous", "amphibologia", "ampliatio", "anacephalaeosis", "anacoenosis", "anacoloutha", "anacoluthon", "anadiplosis", "anamnesis", "anangeon", "anaphora", "anapodoton", "anastrophe", "anemographia", "anesis", "antanaclasis", "antanagoge", "antenantiosis", "anthimeria", "anthropopatheia", "anthypophora", "anticategoria", "anticipation", "antilogy", "antimetabole", "antimetathesis", "antipersonification", "antiphrasis", "antiprosopopoeia", "antiptosis", "antirrhesis", "antisagoge", "antistasis", "antisthecon", "antistrophe", "antithesis", "antitheton", "antonomasia", "apagoresis", "aphaeresis", "aphorismus", "apocarteresis", "apocope", "apodioxis", "apodixis", "apologue", "apophasis", "apoplanesis", "aporia", "aposiopesis", "apostrophe", "apothegm", "apparent refusal", "appositio", "apposition", "ara", "articulus", "aschematismus", "aschematiston", "asphalia", "assonance", "assumptio", "assumption", "asteismus", "astrothesia", "asyndeton", "auxesis", "aversio", "barbarism", "battologia", "bdelygmia", "benedictio", "bomphiologia", "brachiepia", "brachylogia", "cacemphaton", "cacophonia", "cacosyntheton", "cacozelia", "casus pro casu", "catachresis", "catacosmesis", "cataphasis", "cataplexis", "categoria", "characterismus", "charientismus", "chiasmus", "chorographia", "chreia", "chronographia", "circumlocutio", "civille jest", "clause", "climax", "coenotes", "colon", "combined repetition", "comma", "common cause", "commoratio", "communicatio", "commutatio", "comparatio", "compensatio", "complexio", "compositum ex contrariis", "comprobatio", "conceit", "concessio", "conciliatio", "conclusio", "condescensio", "condescension", "conduplicatio", "congeries", "conjunctio", "consonance", "contencion", "contentio", "continued metaphor", "contractio", "contrarium", "contrast", "conversio", "correctio", "counterfait in personation", "counterfait place", "counter turne", "deesis", "dehortatio", "dendrographia", "deprecatio", "descriptio", "diacope", "diaeresis", "dialogismus", "dialysis", "dialyton", "dianoea", "diaphora", "diaporesis", "diaskeue", "diastole", "diasyrmus", "diazeugma", "dicaeologia", "digressio", "dilemma", "dirimens copulatio", "distinctio", "distributio", "ecphonesis", "ecphrasis", "ecthlipsis", "effictio", "elenchus", "ellipsis", "emphasis", "enallage", "enantiosis", "enargia", "encomium", "energia", "enigma", "ennoia", "enthymeme", "enumeratio", "epanalepsis", "epanodos", "epanorthosis", "epenthesis", "epergesis", "epexegesis", "epicrisis", "epilogus", "epimone", "epiphonema", "epiplexis", "epistrophe", "epitasis", "epitheton", "episynaloephe", "epitrochasmus", "epitrope", "epizeugma", "epizeuxis", "erotema", "ethopoeia", "eucharistia", "euche", "eulogia", "euphemismus", "eustathia", "eutrepismus", "example", "excitatio", "exclamatio", "excursus", "exergasia", "exouthenismos", "expeditio", "expolitio", "exuscitatio", "frequentatio", "geographia", "gnome", "graecismus", "hendiadys", "heterogenium", "homiologia", "homoeoprophoron", "homoeosis", "homoioptoton", "homoioteleuton", "horismus", "hydrographia", "hypallage", "hyperbaton", "hyperbole", "hypophora", "hypotyposis", "hypozeugma", "hypozeuxis", "hysterologia", "hysteron proteron", "icon", "indignatio", "inopinaturm", "insinuatio", "interrogatio", "inter se pugnantia", "intimation", "irony", "isocolon", "litotes", "macrologia", "martyria", "maxim", "medela", "meiosis", "membrum", "mempsis", "merismus", "mesarchia", "mesodiplosis", "mesozeugma", "metabasis", "metalepsis", "metallage", "metaphor", "metaplasm", "metastasis", "metathesis", "metonymy", "mimesis", "mycterismus", "noema", "oeonismus", "ominatio", "onedismus", "onomatopoeia", "optatio", "orcos", "oxymoron", "paenismus", "palilogia", "parabola", "paradiastole", "paradiegesis", "paradigma", "paradox", "paraenesis", "paragoge", "paralipsis", "parallelism", "paramythia", "parathesis", "parecbasis", "paregmenon", "parelcon", "parembole", "parenthesis", "pareuresis", "paroemia", "paroemion", "paromoiosis", "paromologia", "paronomasia", "parrhesia", "pathopoeia", "perclusio", "periergia", "period", "periphrasis", "perissologia", "peristasis", "permutatio", "personification", "philophronesis", "pleonasm", "ploce", "polyptoton", "polysyndeton", "pragmatographia", "procatalepsis", "proclees", "prodiorthosis", "proecthesis", "prolepsis", "prosapodosis", "proslepsis", "prosonomasia", "prosopographia", "prosopopoeia", "prosphonesis", "protherapeia", "prothesis", "protrope", "proverb", "prozeugma", "pysma", "ratiocinatio", "repetitio", "repotia", "restrictio", "rhetorical question", "sarcasmus", "scesis onomaton", "schematismus", "scheme", "scurra", "skotison", "sententia", "sermocinatio", "simile", "solecismus", "soraismus", "sorites", "subjectio", "sustentatio", "syllepsis", "syllogismus", "symperasma", "symploce", "synaeresis", "synaloepha", "synathroesmus", "syncatabasis", "syncategorema", "synchoresis", "synchysis", "syncope", "syncrisis", "synecdoche", "synoeciosis", "synonymia", "synthesis", "syntheton", "synzeugma", "systole", "systrophe", "tapinosis", "tasis", "tautologia", "taxis", "thaumasmus", "tmesis", "topographia", "topothesia", "traductio", "transitio", "transplacement", "tricolon", "verborum bombus", "zeugma"]

figures += random.sample(byu_figs,extra_figure_number)

figures = list(set(figures))

len(figures)

58

In [51]:
figures = randrand(figures,min=3) ## sample
len(figures)

39

In [52]:
import re
def COMMENT_recommend_parts(inputtext=None): ## does nothing with argument, but accepts to keep like other functions
  if random.choice([True,True,True,True,False]):
    return_string = "This really needs a %s." % random.choice(figures)
  else:
    return_string = "This would be so much better if it had a %s and a %s." % tuple(random.sample(figures,2))
  return_string = re.sub(r" a ([aeiou])",r" an \1",return_string) ## a -> an
  return return_string

COMMENT_recommend_parts()

'This really needs a chiasmus.'

**Deconstructed Sestina:** One of several words, chosen from a larger list, is suggested as an ending word.

In [53]:
words = ["angel","away","force","city","fortune","boat","art","center","part","earth",
         "face","praise","fruit","shade","rest","motion","store","need","reply",
         "grace","nature","thing","order","frame","wood","crown","bread","tomorrow"] ### some from Herbert

number_of_key_words = random.randrange(1,4)
number_of_key_words = 2
key_words = random.sample(list(set(words)),number_of_key_words)

def COMMENT_suggest_key_word(inputtext=None):
  """
  recommends in order
  """
  word = key_words.pop(0)
  key_words.append(word)
  return 'Try ending with this word: "%s."' % word

In [54]:
for i in range(4):
  print(COMMENT_suggest_key_word())

Try ending with this word: "away."
Try ending with this word: "rest."
Try ending with this word: "away."
Try ending with this word: "rest."


**Google Suggestion:** Suggests Googling a noun and another word and looking for inspiration in the results. 

In [55]:
googlables = [
    "science","1980s","fin de siècle","1970s","Islam","forbidden","theory","Kant","Christ","battle","virtue",
    "fragrance","fabric","house music","downtown scene","the Met","tennis","baseball","automotive","industrial","economic",
    "Nokia","misprision","civil","trad","Brentwood, Tennessee","trivium","McKinsey","Chrysostom","Ashbery","George Herbert","Eno","Princeton","Carpathia"
]

def COMMENT_google_suggestion(text):
  pos_tagged = toktag(text)
  poss_words = [token.lower() for token,tag in pos_tagged if (tag in ["NN","NNS"])]
  if poss_words==[]:
    return None
  else:
    a_noun = random.choice(poss_words)
    a_googlable = random.choice(googlables)
    return_string = ""
    if random.choice([True,False,False]):
      insult_word = random.choice(['uninspired','dull','expected','too familiar'])
      return_string+='A little %s, to be frank. ' % insult_word
    if random.choice([True,False,False,False]):
      return_string+='You need to get outside your own head. '
    if random.choice([True,True,True,False]):
      searchtype = "googling"
    elif random.choice([True,True,False]):
      searchtype = "twitter searching"
    else:
      searchtype = "searching your notes or maybe your email for"
    return_string+='Why don\'t you try %s "%s" and "%s"?' % (searchtype,a_noun,a_googlable)
    if random.choice([True,False]):
      return_string+=' Maybe you\'ll find some inspiration.'
    return return_string

COMMENT_google_suggestion("You are a good friend to me.")

'Why don\'t you try searching your notes or maybe your email for "friend" and "1970s"? Maybe you\'ll find some inspiration.'

**Add Grammatical Moves:** Suggests adding grammatical features (e.g. future tense), testing to make sure input text doesn't already have them.

In [56]:
def future_test(pos_tag):
  regexes = [
      r'going~VBG to~TO', 
      r'will~MD',
  ]
  pos_tagged_as_string = " ".join([token.lower()+"~"+tag for token,tag in pos_tag])
  for reg in regexes:
    if re.search(reg,pos_tagged_as_string)!=None:
      return None
  return "I think this line could be shifted into the future tense."

def future_conditional_test(pos_tag):
  regexes = [
      r'will~MD have~',
  ]
  pos_tagged_as_string = " ".join([token.lower()+"~"+tag for token,tag in pos_tag])
  for reg in regexes:
    if re.search(reg,pos_tagged_as_string)!=None:
      return None
  return "Maybe you could use the future tense."

def conditional_test(pos_tag):
  regexes = [
      r'\w+ould~md'
  ]
  pos_tagged_as_string = " ".join([token.lower()+"~"+tag for token,tag in pos_tag])
  for reg in regexes:
    if re.search(reg,pos_tagged_as_string)!=None:
      return None
  return 'Maybe you could use a word like "could" or "should" to entertain possibilities.'

def past_test(pos_tag):
  regexes = [
      r'\w+~VBD',
      r'\w+~VBN'
  ]
  pos_tagged_as_string = " ".join([token.lower()+"~"+tag for token,tag in pos_tag])
  for reg in regexes:
    if re.search(reg,pos_tagged_as_string)!=None:
      return None
  return 'Try switching this into past tense.'

def proper_test(pos_tag):
  regexes = [
      r'\w+~NNPS?',
  ]
  pos_tagged_as_string = " ".join([token.lower()+"~"+tag for token,tag in pos_tag])
  for reg in regexes:
    if re.search(reg,pos_tagged_as_string)!=None:
      return None
  return 'Maybe a proper noun?'

def personal_test(pos_tag):
  regexes = [
      r'i+~PRP',
      r'me+~PRP',
      r'mine+~PRP',
      r'my+~PRP',
  ]
  pos_tagged_as_string = " ".join([token.lower()+"~"+tag for token,tag in pos_tag])
  for reg in regexes:
    if re.search(reg,pos_tagged_as_string)!=None:
      return None
  return 'Why don\'t you make this a bit more personal, in the first person?'


def proper_test(pos_tag):
  regexes = [
      r'\w+~NNPS?',
  ]
  pos_tagged_as_string = " ".join([token.lower()+"~"+tag for token,tag in pos_tag])
  for reg in regexes:
    if re.search(reg,pos_tagged_as_string)!=None:
      return None
  return 'Maybe a proper noun?'


def preposition_test(pos_tag):
  regexes = [
      r'\w+~IN',
  ]
  pos_tagged_as_string = " ".join([token.lower()+"~"+tag for token,tag in pos_tag])
  for reg in regexes:
    if re.search(reg,pos_tagged_as_string)!=None:
      return None
  return 'I\'d love a prepositional phrase.'


def number_test(pos_tag):
  regexes = [
      r'\w+~CD',
      r'\d',
  ]
  pos_tagged_as_string = " ".join([token.lower()+"~"+tag for token,tag in pos_tag])
  for reg in regexes:
    if re.search(reg,pos_tagged_as_string)!=None:
      return None
  return 'This is a little fuzzy.  It needs a number to make things more precise.'


def comp_super_adj_test(pos_tag):
  regexes = [
      r'\w+~JJR',
      r'\w+~JJS',
  ]
  pos_tagged_as_string = " ".join([token.lower()+"~"+tag for token,tag in pos_tag])
  for reg in regexes:
    if re.search(reg,pos_tagged_as_string)!=None:
      return None
  return random.choice(['What if you included a superlative or comparative adjective? That would make things more intense, wouldn\'t they.',"What if you included a superlative or comparative adjective?"])


tests = [
    future_test,
    conditional_test,
    past_test,
    proper_test,
    preposition_test,
    number_test,
    comp_super_adj_test,
    personal_test,
]

some_tests = randrand(tests,min=2)

def COMMENT_on_grammar_not_used(text,tests=some_tests):
  pos_tagged = toktag(text)

  random.shuffle(tests)
  for t in tests:
    result = t(pos_tagged)
    if result!=None:
      #print(result)
      return result
  return None

In [57]:
COMMENT_on_grammar_not_used("I laugh.")

'This is a little fuzzy.  It needs a number to make things more precise.'

**Line Length:** calls for different line length. 


In [58]:
def COMMENT_on_line_length(text):
  tokens = [tok for tok,tag in toktag(text)]
  ## don't count punctuation
  tokens = [tok for tok in tokens if any(c.isalpha() for c in tok)]
  length = len(tokens)
  possible_delta = round(length * .5)
  delta = random.randrange(1,possible_delta)
  if random.random()<.75:
    delta = delta*-1 ## neg
  goal_length = max(5,length+delta) ## set a minimum length
  return "This is %d words long, but I think the perfect number of words would be...%d." % (length,goal_length)

In [59]:
COMMENT_on_line_length("you aren't---my friend's---dad Mr. Jeff.")

'This is 7 words long, but I think the perfect number of words would be...5.'

**Purposes of Poetry:** Reminds reader of the point of poetry.

In [60]:
purposes = [
 "\"is not heard but overheard\"",
 "is \"the best words in the best order\"",
 "is \"the spontaneous overflow of powerful feelings\"",
 "\"must resist the intelligence almost successfully\"",
 "\"should mean what it says, literally and in every sense\"",
 "\"comes right with a click like a closing box\"",
 "\"gains a maximum of convincing power at the very moment that it abdicates any claim to truth\"",
 "\"is a momentary stay against confusion\"",
 "\"is ontology\"",
 "\"should be palpable and mute / as a globed fruit\"",
 "\"should feel physically as if the top of one's head were taken off\"",
 "\"tells the truth but tell it slant\"",
 "\"is not the sterile word play that, too often, the white fathers distorted the word poetry to mean\"",
]

In [61]:
purpose_of_poetry = random.choice(purposes)
#purpose_of_poetry = purposes[-1]

purpose_of_poetry

'"is ontology"'

In [62]:
def purpose_generator(purpose):
  for i in range(9):
    if i==0:
      resp =  "Just so you know, poetry %s." % purpose
      yield resp.replace('".','."').replace('poetry "is ont','"poetry is ont')
    elif i==1:
      yield "Once again, bear in mind that poetry %s." % purpose.replace('"',"")
    elif i<5: 
      resp = "I still feel like you could do more to make sure that this poem %s." % purpose.replace('"',"")
      resp = resp.replace('should be','is',1).replace('should feel','feels',1).replace('should mean','means',1).replace('must resist','resists',1)
      yield resp
    elif i<8:
      resp = "What is wrong with you? Don't you get that poetry %s?" % purpose.replace('"',"")
      resp = resp.replace("poetry is","poetry should be",1).replace("poetry should be not","poetry should not be").replace("poetry tells","poetry should tell")
      yield resp
    else:
      yield random.choice(["I give up.","...","Oh well, I tried."])

In [63]:
pg = purpose_generator(purpose_of_poetry)

In [64]:
def COMMENT_purpose_of_poetry(inputtext=None):
  try:
    return next(pg)
  except StopIteration:
    return None

In [65]:
for i in range(10):
  print(COMMENT_purpose_of_poetry())

Just so you know, "poetry is ontology."
Once again, bear in mind that poetry is ontology.
I still feel like you could do more to make sure that this poem is ontology.
I still feel like you could do more to make sure that this poem is ontology.
I still feel like you could do more to make sure that this poem is ontology.
What is wrong with you? Don't you get that poetry should be ontology?
What is wrong with you? Don't you get that poetry should be ontology?
What is wrong with you? Don't you get that poetry should be ontology?
I give up.
None


In [66]:
COMMENT_purpose_of_poetry("this is null")

In [67]:
pg = purpose_generator(purpose_of_poetry)

**Extend line:** Offer continuation based on last part-of-speech of input 

In [68]:
tag2next_word = {
    "PRP":["who","who shall","who did","who never","by whom","who could"],
    "VBG":["in","in","up","up","with","without","for","beyond","but not"],
    "VBD":["up","so that","so very"],
    "NN":["that","that will",", the kind","named","made of","with its"],
    "NNS":["that","that together",", all of them so","most of them","of",", which now I describe:",", which now I name:"],
    "JJ":["as the","as one","as those",", if not exactly",", a property yielded by"],
    "NNP":[", a type of",", the only","of",]
}

In [69]:
def extend_line(inputtext,extension):
  ending_part = re.findall(r"(?:\b\w+\b ?){3,4}$",inputtext)
  if ending_part==[]:
    return None
  else:
    ending_part = ending_part[0]
  output =  "Yes! Keep going: \"...%s %s...\"" % (ending_part,extension)
  output = output.replace(" ,",",")
  return output

In [70]:
def COMMENT_extend_line(inputtext):
  inputtext = inputtext.rstrip(".,?\"!")
  pos_tagged = toktag(inputtext)
 #print(pos_tagged)
  # while pos_tagged[-1][1][0].isalpha()==False: ## while last token is punctuation
  #   pos_tagged.pop()
  last_pos = pos_tagged[-1][1]
  if last_pos not in tag2next_word:
    return None
  extension = random.choice(tag2next_word[last_pos])
  return extend_line(inputtext,extension)

In [71]:
COMMENT_extend_line("Take no light unless it be perfect.")

'Yes! Keep going: "...unless it be perfect, a property yielded by..."'

**Push Toward Common Words:** Find a word that is rare and suggest the writer supply a plain, common alternative.

In [72]:
from nltk.corpus import brown,gutenberg
from nltk import FreqDist
import random

In [73]:
gutenwords_all = [w.lower() for w in gutenberg.words()]
brownwords_all = [w.lower() for w in brown.words()]
brown_gut_freqdist = FreqDist(gutenwords_all+brownwords_all)

In [74]:
brown_gut_freqdist["dog"]

189

In [75]:
min_count_of_word = random.randrange(1,4) ## 1 2 or 3
min_count_of_word

1

In [76]:
def get_count(token,the_fd=brown_gut_freqdist):
  if token not in brown_gut_freqdist:
    return 0
  else:
    return brown_gut_freqdist[token]

In [77]:
from nltk.parse.nonprojectivedependencyparser import nonprojective_conll_parse_demo
def COMMENT_scold_rare_word(inputtext):
  toks = [tok.lower() for tok,tag in toktag(inputtext) if tag[0] in "NJV"]
  tok_count = [(tok,get_count(tok)) for tok in toks]
  #print(tok_count)
  rare_toks = [t for t,c in tok_count if c<min_count_of_word]
  critique_word = random.choice(["affected","pretentious"])
  virtue_word = random.choice(["common","plain","unassuming","simple","simple and confident"])
  if len(rare_toks)==0:
    return None
  else:
    rare_toks[0]=rare_toks[0].title()
    if len(rare_toks)==1:
      if random.random()>.5:
        return "\"%s\" is a bit too %s.  Why not something more %s?" % (rare_toks[0],critique_word,virtue_word)
      else:
        return "\"%s\" is a bit too %s.  A rare word does not make a rare poem." % (rare_toks[0],critique_word)
    else:
      return "\"%s\"---a bit %s, no?" % (",\" \"".join(rare_toks),critique_word)

  
COMMENT_scold_rare_word("I will adminadvert against my infelicitious fate.")

'"Adminadvert," "infelicitious"---a bit pretentious, no?'

## Meta-function

Combine individual functions into writing interface.

In [78]:
import inspect
import types
from os.path import join

def is_local(object):
    return isinstance(object, types.FunctionType) and object.__module__ == __name__

import sys
funcs = [value for name, value in inspect.getmembers(sys.modules[__name__], predicate=is_local) if name.startswith("COMMENT_")]
funcs

[<function __main__.COMMENT_NN_recommendation(line)>,
 <function __main__.COMMENT_ban_pos(text)>,
 <function __main__.COMMENT_chunk_recommendation(text)>,
 <function __main__.COMMENT_extend_line(inputtext)>,
 <function __main__.COMMENT_fix_a_word(text)>,
 <function __main__.COMMENT_flip(text)>,
 <function __main__.COMMENT_google_suggestion(text)>,
 <function __main__.COMMENT_on_grammar_not_used(text, tests=[<function number_test at 0x7f4e99ea0790>, <function future_test at 0x7f4e99ea0310>, <function conditional_test at 0x7f4e99ea0ee0>, <function preposition_test at 0x7f4e99ea0670>, <function comp_super_adj_test at 0x7f4e99ea0820>, <function personal_test at 0x7f4e99ea0430>])>,
 <function __main__.COMMENT_on_line_length(text)>,
 <function __main__.COMMENT_purpose_of_poetry(inputtext=None)>,
 <function __main__.COMMENT_recommend_parts(inputtext=None)>,
 <function __main__.COMMENT_remove_letter(text)>,
 <function __main__.COMMENT_replace_word(text)>,
 <function __main__.COMMENT_scold_rare

In [79]:
len(funcs)

19

In [80]:
faces = "🙋🏾‍♂️ 👨🏼‍🎨 🙎🏻‍♂️ 🙎🏼‍♀️ 🤦🏾  🤦🏻‍♀️ 👨🏻‍🌾 🕵🏻‍♀️ 🧝🏿‍♂️ 🙍🏻‍♀️ 👨🏼‍💻 🙅🏼‍♂️ 🧕🏼 🧑🏽‍💼 🧑🏼‍🏫 👨🏽‍💼 👨🏿‍🎤 👩🏽‍💼 🤵🏻 💁🏼‍♂️ 💁🏽‍♀️".split()
func2face = {func.__name__:face for func,face in list(zip(funcs,faces[:len(funcs)]))}

In [81]:
funcs_sample = randrand(funcs,min=10)
funcs_sample

[<function __main__.COMMENT_extend_line(inputtext)>,
 <function __main__.COMMENT_with_wikipedia(text)>,
 <function __main__.COMMENT_recommend_parts(inputtext=None)>,
 <function __main__.COMMENT_suggest_rhyme(text)>,
 <function __main__.COMMENT_test_against_banned_topics(text, banned_topics=['industrial', 'body', 'face', 'religion', 'writing', 'finance', 'political', 'shopping', 'lustrous', 'darkness', 'music', 'romance', 'death', 'sport', 'school', 'furniture', 'kinfolk', 'nature', 'food', 'digital', 'negative', 'urban', 'violent', 'ocean', 'bird', 'celestial', 'architectural', 'war', 'science'])>,
 <function __main__.COMMENT_scold_rare_word(inputtext)>,
 <function __main__.COMMENT_remove_letter(text)>,
 <function __main__.COMMENT_replace_word(text)>,
 <function __main__.COMMENT_ban_pos(text)>,
 <function __main__.COMMENT_chunk_recommendation(text)>,
 <function __main__.COMMENT_NN_recommendation(line)>]

In [82]:
faces_sample = [func2face[f.__name__] for f in funcs_sample]

In [83]:
repeated_funcs = []

for f in funcs_sample:
  times_to_repeat = random.randint(0,50)
  temp_f = f
  for i in range(times_to_repeat):
    repeated_funcs.append(temp_f)

funcs_sample_multiplied = funcs_sample+repeated_funcs
len(funcs_sample_multiplied)

287

In [84]:
random.shuffle(funcs_sample_multiplied)

In [85]:
print("~L O T U S C H O R U S W O R K S H O P~")
print("")
print("                 🌸")
print("a γυμνάσιον")
print("")
print("*"*39)
print("")
print ("INSTRUCTIONS:")
print (" - write a sentence of poetry")
print (" - receive feedback")
print (" - revise your sentence accordingly")
print (" - repeat")
print (' - type "quit" to quit')
print("")
print("*"*39)

max_comments = random.randrange(2,8)
max_comments = 8

seen_line = False

first_responses = ["Well revised...now keep going, write a new sentence.","Good. Now write another sentence."]
first_responses.append("See how your verse becomes more %s?" % random.choice(['true','dense','perfect','elemental','cerebral','muscular','liberated','ordered','like you','unlike you','immortal and free']))
first_responses.append("You have earned this symbol of your %s: 🌸" % random.choice(['growth','selflessness','openness','purity']))
first_responses = ["   👤:"+i for i in first_responses]

recent_funcs = [] ## will be list of lists
cool_off = 2 ## times after using a function it will be unavailable

how_critical = [True]+[False]*random.randint(1,10) ## how often it will be satisfied without changes

while True:
  comments = []
  used_funcs = []
  user_input=""
  ## handle non-input
  while user_input=="":
    if seen_line==False:
      user_input = input(">")
    else:
      user_input = input(">")
  if user_input.lower() in ["quit","exit"]:
    break
  if (seen_line==False): ## only offer feedback once
    if (random.choice(how_critical) and len(first_responses)==0):
      print("   👤:Not bad.  Go on.")
      print("")
    else:
      seen_line = True # flip
      random.shuffle(funcs)
      ## print(recent_funcs)
      recent_funcs_flat = list(itertools.chain(*recent_funcs))
      for f in funcs_sample_multiplied: 
        #print(f)
        if (f not in used_funcs and f not in recent_funcs_flat): 
          result = f(user_input)
          if result!=None:
            result = func2face[f.__name__]+":"+result
            comments.append(result)
            used_funcs.append(f)
          if len(comments)==max_comments:
            break
          if (len(comments)>0 and random.choice([True,False,False,False,False,False,False])): ## don't always to go max
            break
      ## don't always print max 
      # if len(comments)!=0:
      #   comments_maybe_not_all = [comments[0]] ## always print 1st one
      #   for i in comments[1:]: ## maybe add the rest
      #     if random.choice([True,False,False,False]):
      #       comments_maybe_not_all.append(i)
      #     else:
      #       break
      ## print out
      if len(comments)==0:  ## fallback if no successful comments
        if random.choice([True,True,False]):
          print("   👤:You must defeat yourself by becoming more like yourself.")
        else:
          seen_line = False
          print("   👤:Good enough.  Go on.")
          print("")
      else: ## typical
        for c in comments:
          print("   %s" % c)
      ## keep track of used functions
      recent_funcs.append(used_funcs)
      if len(recent_funcs)>cool_off:
        recent_funcs = recent_funcs[-cool_off:] ## get last n
  else:
    seen_line=False
    if len(first_responses)!=0:
      response = first_responses.pop(0)
    else:
      if random.choice([True,True,False]):
        response = "   👤:"+random.choice(["Good.","Much better.","Yes, better.","Nice","👍","Well revised.","Go on...","Go on..."])
      else:
        gift = "👤:For your efforts, please take this: %s" % random.choice("🌼 🌸 🌸 🌸 🌸 🌸 💐 🌺 🌷 🌻 🥀".split(" "))
        response = "   "+gift
    print(response)
    print("")

~L O T U S C H O R U S W O R K S H O P~

                 🌸
a γυμνάσιον

***************************************

INSTRUCTIONS:
 - write a sentence of poetry
 - receive feedback
 - revise your sentence accordingly
 - repeat
 - type "quit" to quit

***************************************


KeyboardInterrupt: ignored

***